In [48]:
import pandas as pd
import numpy as np
import matplotlib as plt
from itertools import cycle
from sqlalchemy import create_engine

conexion a base de datos

In [17]:
usuario = 'ccornejo'
contrasena = '12345678'
url_servidor = '10.8.64.44'
puerto = '3306'
esquema = 'siscont'


In [18]:
c_conexion = 'mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}'
c_conexion = c_conexion.format(usuario, contrasena, url_servidor, puerto, esquema                               )


In [19]:
motor_mysql_mariadb = create_engine(c_conexion)

In [23]:
consulta_sql = 'select * from vw_pacientes'


In [24]:
datos = pd.read_sql_query(consulta_sql, motor_mysql_mariadb)

In [26]:
sic = datos

In [39]:
def digito_verificador(rut):
    reversed_digits = map(int, reversed(str(rut)))
    factors = cycle(range(2, 8))
    s = sum(d * f for d, f in zip(reversed_digits, factors))
    dv = (-s) % 11
    if dv == 11: dv=0
    elif dv == 10: dv="K"
    return dv

In [40]:
def reemplazamayuscula(dv):
    return dv.upper()

In [27]:
sic.shape

(771494, 26)

In [28]:
sic.columns

Index(['id', 'tipoDoc', 'rut', 'dv', 'numDoc', 'nombre', 'apPaterno',
       'apMaterno', 'fechaNacimiento', 'genero', 'prevision', 'tramo', 'prais',
       'funcionario', 'vias', 'direccion', 'numero', 'x', 'y', 'codcomuna',
       'comuna', 'telefono', 'telefono2', 'email', 'created_at', 'updated_at'],
      dtype='object')

Formato

In [29]:
sic.dtypes

id                          int64
tipoDoc                     int64
rut                       float64
dv                         object
numDoc                     object
nombre                     object
apPaterno                  object
apMaterno                  object
fechaNacimiento            object
genero                     object
prevision                  object
tramo                      object
prais                      object
funcionario                 int64
vias                       object
direccion                  object
numero                     object
x                         float64
y                         float64
codcomuna                  object
comuna                     object
telefono                   object
telefono2                  object
email                      object
created_at         datetime64[ns]
updated_at         datetime64[ns]
dtype: object

In [30]:
sic['rut'].isnull().sum(axis=0)

1353

In [31]:
sic['dv'].isnull().sum(axis=0)

1374

In [33]:
sic['run'] = sic['rut'].fillna(0)

In [34]:
sic['dv'] = sic['dv'].fillna('SIN DV')

Cambio tipo de datos.

In [35]:
sic['run'] = sic['run'].astype('int64') 

In [36]:
sic['dv'] = sic['dv'].astype('str')

In [37]:
sic.columns.str.lower().str.replace(' ', '')

Index(['id', 'tipodoc', 'rut', 'dv', 'numdoc', 'nombre', 'appaterno',
       'apmaterno', 'fechanacimiento', 'genero', 'prevision', 'tramo', 'prais',
       'funcionario', 'vias', 'direccion', 'numero', 'x', 'y', 'codcomuna',
       'comuna', 'telefono', 'telefono2', 'email', 'created_at', 'updated_at',
       'run'],
      dtype='object')

In [41]:
sic['dv'] = sic['dv'].apply(reemplazamayuscula)

In [42]:
sic['dvrev'] = sic['run'].apply(digito_verificador)

In [43]:
sic['dvrev'] = sic['dvrev'].astype('str')

In [44]:
sic.dtypes

id                          int64
tipoDoc                     int64
rut                       float64
dv                         object
numDoc                     object
nombre                     object
apPaterno                  object
apMaterno                  object
fechaNacimiento            object
genero                     object
prevision                  object
tramo                      object
prais                      object
funcionario                 int64
vias                       object
direccion                  object
numero                     object
x                         float64
y                         float64
codcomuna                  object
comuna                     object
telefono                   object
telefono2                  object
email                      object
created_at         datetime64[ns]
updated_at         datetime64[ns]
run                         int64
dvrev                      object
dtype: object

## definir condicionales

In [45]:
sic['comparacion'] = np.where(sic['dv']==sic['dvrev'], 'OK', 'ERROR')

In [ ]:
sic.to_excel('revisado.xlsx')